#Funções

In [18]:
!pip install emoji
!pip install unidecode

In [19]:
import re
import joblib
import pandas as pd
from scipy.sparse import hstack
import pickle
import sys
import emoji
from unidecode import unidecode

In [20]:
def limpaTexto(text):
    # Remover emojis
    text = emoji.replace_emoji(text, replace='')  # Remove emojis

    # Remover menções de usuários
    text = re.sub(r'@\w+', ' ', text)

    # Remover links
    text = re.sub(r'https?://\S+', ' ', text)

    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text)

    # Remover palavras específicas
    text = re.sub(r'\b(rt|user|https)\b', ' ', text, flags=re.IGNORECASE)

    # Remover acentuação
    text = unidecode(text)

    return text.strip()

def preprocessaTexto(text):
    text = limpaTexto(text)
    return text

# Carregar listas de palavras tóxicas e não tóxicas usando pickle
url_toxic = 'https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/toxic_words.pkl'
!wget {url_toxic} -O toxic_words.pkl

url_nt = 'https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/non_toxic_words.pkl'
!wget {url_toxic} -O non_toxic_words.pkl

with open('toxic_words.pkl', 'rb') as file:
    toxic_words = pickle.load(file)

with open('non_toxic_words.pkl', 'rb') as file:
    non_toxic_words= pickle.load(file)

# Funções de extração de features
def contaToxicos(text, toxic_words):
  count = 0
  for p in text.split():
    if p in toxic_words:
      count += 1
  return count

def contaNaotoxicos(text, nontoxic_words):
  count = 0
  for p in text.split():
    if p in nontoxic_words:
      count += 1
  return count

def contaNeutros(text, toxic_words, nontoxic_words):
  words = text.split()
  count = 0
  for p in text.split():
    if p in nontoxic_words and p in toxic_words:
      count += 1
  return count

def contemToxicos(text, toxic_words):
  for p in text.split():
    if p in toxic_words:
      return 1
  return 0

def contemNaotoxicos(text, nontoxic_words):
  for p in text.split():
    if p in nontoxic_words:
      return 1
  return 0

def contemNeutros(text, toxic_words, nontoxic_words):
  for p in text.split():
     if p in nontoxic_words and p in toxic_words:
      return 1
  return 0


# Carregar vetorizador e modelo
url_veto = 'https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/vectorizer.pkl'
!wget {url_veto} -O vectorizer.pkl
vetorizador = joblib.load('vectorizer.pkl')

url_model = 'https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/identifica_toxicos.pkl'
!wget {url_model} -O model.pkl
modelo = joblib.load('model.pkl')

# Função principal para prever a toxicidade de um tweet
def predizToxico(tweet):
    # Pré-processar o texto
    tweet = preprocessaTexto(tweet)

    # Criar features
    features = {
        'text': tweet,
        'toxic_words': contemToxicos(tweet, toxic_words),
        'cont_toxic_words': contaToxicos(tweet, toxic_words),
        'non_toxic_words': contemNaotoxicos(tweet, non_toxic_words),
        'cont_non_toxic_words': contaNaotoxicos(tweet, non_toxic_words),
        'neutral_words': contemNeutros(tweet, toxic_words, non_toxic_words),
        'cont_neutral_words': contaNeutros(tweet, toxic_words, non_toxic_words),
        'char_cont': len(tweet),
        'words_cont': len(tweet.split())
    }

    # Transformar texto usando TF-IDF
    X_t = vetorizador.transform([tweet])

    # Criar DataFrame com outras features
    X_others = pd.DataFrame([features]).drop(columns=['text'])

    # Combinar as features
    X = hstack([X_t, X_others])

    # Fazer a predição
    prediction = modelo.predict(X)

    return prediction[0]

--2024-05-31 00:17:22--  https://github.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/raw/main/toxic_words.pkl
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/toxic_words.pkl [following]
--2024-05-31 00:17:22--  https://raw.githubusercontent.com/VictorHNascimento/ML-Olympiad-Toxic-Language-PTBR-Detection/main/toxic_words.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212765 (208K) [application/octet-stream]
Saving to: ‘toxic_words.pkl’

toxic_words.pkl     100%[===================>] 207.78K  --.-KB/s    in 0.

#Aplicação

In [28]:
# Função principal para entrada do usuário
x = 1
while x == 1:
  if __name__ == "__main__":
      tweet = input("Digite o tweet: ")
      prediction = predizToxico(tweet)
      if prediction == 1:
          print("\033[1;31mO tweet é tóxico\033[0m")
      else:
          print("\033[1;32mO tweet é não tóxico\033[0m")

      print()
      x = int(
          input('Digite 1 se deseja fazer uma nova predição: ')
      )
      print()

Digite o tweet: Vai se foder @user, caralho, se mata
O tweet é tóxico

Digite 1 se deseja fazer uma nova predição: 1

Digite o tweet: Acabei de receber um: 'assistia teus vídeos desde quando eu era criança' na academia de um cara com 3x mais braço que eu, fiquei triste
O tweet é não tóxico

Digite 1 se deseja fazer uma nova predição: 0

